## 路径表示  
PurePosixPath 和 PureWindowsPath 可以实例化于任何的操作系统，仅仅用做处理路径名称。如果要实例化一个与真实文件系统相关的具体类，应该使用 Path, 它会根据当前的操作系统类型得到 PosixPath 或者 WindowsPath 的实例。 

## 构建路径

In [3]:
import pathlib

usr = pathlib.PurePosixPath('usr')
print(usr)

lib = usr / ('lib')
print(lib)

usr_share = usr / pathlib.PurePosixPath('share')
print(usr_share)

root = usr / '..'
print(root)

etc = root / '/etc/'
print(etc)

usr = pathlib.PurePosixPath('/usr')
print(usr)

root = usr / '..'  # 从运行结果看，其仅仅将路径组合起来，而没有去解析 
print(root)

etc = root / '/etc/'
print(etc)

usr
usr/lib
usr/share
usr/..
/etc
/usr
/usr/..
/etc


In [11]:
usr_local = pathlib.Path('/usr1ss/local1')
share = usr_local / '..' / 'share'
print(share)
print(share.resolve())   # 仅生成绝对路径

/usr1ss/local1/../share
/usr1ss/share


In [12]:
root = pathlib.PurePosixPath('/')
subdirs = ['usr', 'local', 'tt']
usr_local = root.joinpath(*subdirs)
print(usr_local)

/usr/local/tt


In [13]:
ind = pathlib.PurePosixPath('source/pathlib/index.rst')
print(ind)

py = ind.with_name('pathlib_from_existing.py')
print(py)

pyc = ind.with_suffix('.pyc')
print(pyc)

source/pathlib/index.rst
source/pathlib/pathlib_from_existing.py
source/pathlib/index.pyc


## 解析路径

In [20]:
p = pathlib.PurePosixPath('/usr/local/sth/demo.py')
print(p.parts)

('/', 'usr', 'local', 'sth', 'demo.py')


In [21]:
print(p.parent)

/usr/local/sth


In [22]:
for u in p.parents:
    print(u)

/usr/local/sth
/usr/local
/usr
/


In [23]:
print(p)
print(p.name)
print(p.suffix)
print(p.stem)

/usr/local/sth/demo.py
demo.py
.py
demo


## 创建具体路径

In [25]:
home = pathlib.Path.home()
print(home)
cwd = pathlib.Path.cwd()
print(cwd)

/Users/hejl
/Users/hejl/local/practise/source_code/standard_library


## 目录内容

In [45]:
p = pathlib.Path('..')
for f in p.iterdir():
    print(f)

../Werkzeug.py
../FlaskOrigin.py
../.ipynb_checkpoints
../testLocalStack.py
../standard_library
../note_of_python_learnnote


In [47]:
for f in p.glob('*.py'):
    print(f)

../Werkzeug.py
../FlaskOrigin.py
../testLocalStack.py


In [54]:
for f in p.rglob('p*.ipynb'):   # rglob(*.py) == glob(**/*.py) 递归查找
    print(f)

../standard_library/pathlib.ipynb
../standard_library/.ipynb_checkpoints/pathlib-checkpoint.ipynb


## 读写文件

In [5]:
f = pathlib.Path('example.txt')
f.write_bytes(b"hello world")

11

In [8]:
with f.open('r', ) as handle:
    print(f"read from open: {handle.read()!r}")

read from open: 'hello world'


In [7]:
print(f"read_text: {f.read_text()!r}")

read_text: 'hello world'


## 操作目录和符号连接

In [9]:
p = pathlib.Path('example_dir')

In [10]:
p.mkdir()

In [11]:
p.mkdir()

FileExistsError: [Errno 17] File exists: 'example_dir'

In [14]:
p = pathlib.Path('example_link')
p.symlink_to('index.rst')  # 使用 symlink_to() 创建一个符号链接

In [15]:
print(p)
print(p.resolve())  # 然后用 resolve() 方法读取符号链接指向的目标的名称
print(p.resolve().name)

example_link
/Users/hejl/local/practise/source_code/standard_library/index.rst
index.rst


## 文件类型

In [29]:
import itertools
import os
root = pathlib.Path('test_files')
if root.exists():
    for f in root.iterdir():
        f.unlink()
else:
    root.mkdir()
# 创建测试文件
(root / 'file').write_text('hello, this is a test file')
(root / 'symlink').symlink_to('file')
os.mkfifo(str(root/'fifo'))

to_scan = itertools.chain(root.iterdir(), [pathlib.Path('/dev/disk0'), pathlib.Path('/dev/console')])
print(f"{'Name':18s} {'File':^5} {'Dir':^5} {'Link':>5} {'FIFO':>5} {'Block':>5} {'Character':^5}")
for f in to_scan:
    print(f"{str(f):18s} {f.is_file():^5} {f.is_dir():^5} {f.is_symlink():^5} {f.is_fifo():^5} {f.is_block_device():^5} {f.is_char_device():^5}")

Name               File   Dir   Link  FIFO Block Character
test_files/symlink   1     0     1     0     0     0  
test_files/file      1     0     0     0     0     0  
test_files/fifo      0     0     0     1     0     0  
/dev/disk0           0     0     0     0     1     0  
/dev/console         0     0     0     0     0     1  


## 文件属性

In [34]:
import time
p = pathlib.Path('example.txt')
stat_info = p.stat()
print(p,':')
print(f' Size:{stat_info.st_size}')
print(f' Permessions:{oct(stat_info.st_mode)}')
print(f' Owner:{stat_info.st_uid}')
print(f' Owner:{p.owner()}')
print(f' Group:{p.group()}')
print(f' Device:{stat_info.st_dev}')
print(f' Created:{time.ctime(stat_info.st_ctime)}')
print(f' Last Modified:{time.ctime(stat_info.st_mtime)}')
print(f' Last Accessed:{time.ctime(stat_info.st_atime)}')

example.txt :
 Size:11
 Permessions:0o100644
 Owner:501
 Owner:hejl
 Group:staff
 Device:16777220
 Created:Sat Sep 22 18:48:40 2018
 Last Modified:Sat Sep 22 18:48:40 2018
 Last Accessed:Sat Sep 22 20:01:11 2018


stat() 方法只是返回了数字类型的系统 ID ，但是 owner() 和 group() 会查找与 ID 相关的名称。

In [39]:
p = pathlib.Path('touch')

if p.exists():
    print('already have this file')
else:
    print('creat a file')
p.touch()
start = p.stat()

time.sleep(1)

p.touch()  # 多次运行touch更新文件的时间
end = p.stat()

print('Start:', time.ctime(start.st_mtime))
print('End  :', time.ctime(end.st_mtime))

already have this file
Start: Sat Sep 22 21:37:50 2018
End  : Sat Sep 22 21:37:51 2018


## 权限

In [45]:
import stat
import os

# 创建一个全新的测试文件
f = pathlib.Path('pathlib_chmod_example.txt')
if f.exists():
    f.unlink()
f.write_text('contents')

# 使用 stat 方法来检测文件权限
existing_permissions = stat.S_IMODE(f.stat().st_mode)
print('Before: {:o}'.format(existing_permissions))

# 决定用啥方法来处理
if not (existing_permissions & os.X_OK):
    print('Adding execute permission')
    new_permissions = existing_permissions | stat.S_IXUSR
else:
    print('Removing execute permission')
    # 使用 xor 来移除用户可执行权限
    new_permissions = existing_permissions ^ stat.S_IXUSR

# 修改权限，并打印修改后的权限结果
f.chmod(new_permissions)
after_permissions = stat.S_IMODE(f.stat().st_mode)
print('After: {:o}'.format(after_permissions))

Before: 644
Adding execute permission
After: 744


## 删除

In [48]:
p = pathlib.Path('example_dir')
p.mkdir()
p.rmdir()

In [49]:
p.rmdir()

FileNotFoundError: [Errno 2] No such file or directory: 'example_dir'

In [50]:
p.mkdir()
(p/'aa.txt').touch()
p.rmdir()

OSError: [Errno 66] Directory not empty: 'example_dir'

rmdir()只能移除空目录，不存在或其内有文件时会抛出不同的异常

In [51]:
(p/'aa.txt').unlink()
print(f"file exists ? {(p/'aa.txt').exists()}")

file exists ? False


In [52]:
(p/'aa.txt').unlink()

FileNotFoundError: [Errno 2] No such file or directory: 'example_dir/aa.txt'